## Notebook for creating AI for playing Skat

#### Importing needed libraries

In [3]:
import gym
from gym import spaces
from pydealer import Card, Deck, SUITS, Stack, const

## Creating Enviroment for SKAT card game 

Full rules can be found here: https://www.pagat.com/schafkopf/skat.html

I'll explain brifly the rules:

1. Deciding the solo player

    a. solo player decides wheather to take the 2 left cards or not
    
    b. Solo players swaps discards 2 cards back into the skat (other players don't know what cards solo player discarded) - those cards also count for the solo player score
    
    c. Solo player anouces the type of the game played

2. Untill all players have cards:
    
    a. Active player plays a card from the hand
    
    b. player on the left plays next card 
    
    c. last player plays a card 
    
    d. Player with the highest value takes the cards, the value is detrmined by colour and value of the cards

3. Players sum up scores - solo player by his cards and comperes with the score of the rest
4. Determination if the solo player has won or not


In [17]:
import pydealer
def create_skat_deck():
    deck = Deck()
    stack = Stack()
    # stack = []

    for i in range(0,52):
        card = deck.deal(1)
        if card.cards[0] >= lowest_card:
            stack.add(card.cards[0])
        else:
            pass
    stack
    # new_deck = pydealer.deck.Deck(stack)
    return stack

deck_skat = create_skat_deck()
# Add the cards to the de
# deck_skat = Deck(stack)
# Print the deck
print(deck_skat)
# print(len(stack))
# print(stack)

Ace of Spades
Ace of Hearts
Ace of Clubs
Ace of Diamonds
King of Spades
King of Hearts
King of Clubs
King of Diamonds
Queen of Spades
Queen of Hearts
Queen of Clubs
Queen of Diamonds
Jack of Spades
Jack of Hearts
Jack of Clubs
Jack of Diamonds
10 of Spades
10 of Hearts
10 of Clubs
10 of Diamonds
9 of Spades
9 of Hearts
9 of Clubs
9 of Diamonds
8 of Spades
8 of Hearts
8 of Clubs
8 of Diamonds
7 of Spades
7 of Hearts
7 of Clubs
7 of Diamonds


In [18]:
class Skat:
    def __init__(self):
        self.deck = create_skat_deck()
        self.deck.shuffle()
    def deal_player(self):
        return self.deck.deal(10)
    def deal_skat(self):
        return self.deck.deal(2)
    def rebulid(self):
        self.deck = create_skat_deck()
        self.deck.shuffle()
        

In [19]:
class Player:
    def __init__(self,skat):
        self.hand = skat.deal_player()
        self.score = 0

        ### Pseudo code from here
    def play_card(self,card):
        if card in self.hand:
            return deal(card)
        else:
            print("trying to play card that does not belong to player's hand")
            return
        

In [7]:
class Blackjack:
    def __init__(self):
        self.deck = Deck()
        self.deck.shuffle()
        self.player_hand = []
        self.dealer_hand = []

    def deal_cards(self):
        self.player_hand = [self.deck.deal() for _ in range(2)]
        self.dealer_hand = [self.deck.deal() for _ in range(2)]

    def hit(self, hand):
        hand.append(self.deck.deal())

    def calculate_hand_value(self, hand):
        hand_value = sum(card.value for card in hand)
        num_aces = sum(card.value == 11 for card in hand)

        while hand_value > 21 and num_aces > 0:
            hand_value -= 10
            num_aces -= 1

        return hand_value

    def player_turn(self):
        while True:
            hand_value = self.calculate_hand_value(self.player_hand)
            print("Player's hand:", self.player_hand)
            print("Player's hand value:", hand_value)

            if hand_value > 21:
                print("Player busts!")
                return -1

            action = input("Do you want to hit or stand? ")
            if action == "hit":
                self.hit(self.player_hand)
            else:
                return hand_value

    def dealer_turn(self):
        while True:
            hand_value = self.calculate_hand_value(self.dealer_hand)
            print("Dealer's hand:", self.dealer_hand)
            print("Dealer's hand value:", hand_value)

            if hand_value >= 17:
                return hand_value

            self.hit(self.dealer_hand)

    def play(self):
        self.deal_cards()
        player_hand_value = self.player_turn()

        if player_hand_value == -1:
            return -1

        dealer_hand_value = self.dealer_turn()

        if dealer_hand_value == -1 or dealer_hand_value > player_hand_value:
            return 0
        elif dealer_hand_value < player_hand_value:
            return 1
        else:
            return -1


In [10]:
deck= Deck()

In [11]:
deck.is_empty()

AttributeError: 'Deck' object has no attribute 'is_empty'

In [12]:
len(deck)

52

11 of Spades
12 of Spades
9 of Clubs
13 of Hearts
13 of Spades
9 of Spades
10 of Clubs
7 of Hearts
8 of Spades
10 of Spades
13 of Diamonds
11 of Clubs
10 of Diamonds
8 of Diamonds
8 of Clubs
7 of Spades
11 of Hearts
12 of Clubs
12 of Hearts
9 of Hearts
7 of Diamonds
12 of Diamonds
9 of Diamonds
13 of Clubs
10 of Hearts
8 of Hearts
7 of Clubs
11 of Diamonds
28


In [8]:
game = Blackjack()
result = game.play()

if result == -1:
    print("It's a tie!")
elif result == 0:
    print("Dealer wins!")
else:
    print("Player wins!")


AttributeError: 'Stack' object has no attribute 'value'

In [5]:
class BlackjackEnv(gym.Env):
    def __init__(self):
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(10),
            spaces.Discrete(10),
            spaces.Discrete(2)
        ))
        self.reset()

    def reset(self):
        self.game = Blackjack()
        self.game.deal_cards()
        self.player_hand_value = self.game.calculate_hand_value(self.game.player_hand)
        self.dealer_card = self.game.dealer_hand[0]
        return (self.player_hand_value, self.dealer_card.value, False)

    def step(self, action):
        if action == 1:
            self.game.hit(self.game.player_hand)
            self.player_hand_value = self.game.calculate_hand_value(self.game.player_hand)
            if self.player_hand_value > 21:
                reward = -1
                done = True
            else:
                reward = 0
                done = False
        else:
            while self.game.calculate_hand_value(self.game.dealer_hand) < 17:
                self.game.hit(self.game.dealer_hand)
            dealer_hand_value = self.game.calculate_hand_value(self.game.dealer_hand)
            if dealer_hand_value > 21 or dealer_hand_value < self.player_hand_value:
                reward = 1
            elif dealer_hand_value == self.player_hand_value:
                reward = 0
            else:
                reward = -1
            done = True

        return (self.player_hand_value, self.dealer_card.value, done), reward, done, {}

    def render(self, mode="human"):
        print("Player's hand:", self.game.player_hand)
        print("Player's hand value:", self.player_hand_value)
        print("Dealer's hand:", self.game.dealer_hand)



In [6]:
env = BlackjackEnv()

AttributeError: 'Stack' object has no attribute 'value'

In [ ]:
state = env.reset()
done = False
score = 0 
while not done:
    env.render()
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    score+=reward
print('Score:{}'.format(score))
env.close()